In [1]:
pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-awxjvu8h
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-awxjvu8h
  Resolved https://github.com/huggingface/transformers to commit 07998ef39926b76d3f6667025535d0859eed61c3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.8 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.33.0.dev0-py3-none-any.whl size=7604296 sha256=6189e407a3e8ee1f8cc589c7ec622b09148c2789104770d7ba65486837f7b717
  Stored in directory: /tmp/pip-ephem-wheel-cache-n19qbsla/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully

In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00


In [4]:
from datasets import load_dataset

minds = load_dataset("PolyAI/minds14", name="en-AU", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
from transformers import pipeline

classifier = pipeline(
    "audio-classification",
    model="anton-l/xtreme_s_xlsr_300m_minds14",
)

In [6]:
classifier(minds[0]["path"])

[{'score': 0.9623644948005676, 'label': 'pay_bill'},
 {'score': 0.028678642585873604, 'label': 'freeze'},
 {'score': 0.0034296344965696335, 'label': 'card_issues'},
 {'score': 0.0020604985766112804, 'label': 'abroad'},
 {'score': 0.0008625703048892319, 'label': 'high_value_payment'}]

In [7]:
speech_commands = load_dataset(
    "speech_commands", "v0.02", split="validation", streaming=True
)

sample = next(iter(speech_commands))

In [8]:
classifier = pipeline(
    "audio-classification", model="MIT/ast-finetuned-speech-commands-v2"
)
classifier(sample["audio"])

[{'score': 0.9999892711639404, 'label': 'backward'},
 {'score': 1.7504938796264469e-06, 'label': 'happy'},
 {'score': 6.703040185129794e-07, 'label': 'follow'},
 {'score': 5.805890168630867e-07, 'label': 'stop'},
 {'score': 5.614535893982975e-07, 'label': 'up'}]

In [10]:
from IPython.display import Audio

classifier(sample["audio"].copy())
Audio(sample["audio"]["array"], rate=sample["audio"]["sampling_rate"])

ValueError: ignored

In [13]:
fleurs = load_dataset("google/fleurs", "all", split="validation", streaming=True)
sample = next(iter(fleurs))

In [15]:
classifier = pipeline(
    "audio-classification", model="sanchit-gandhi/whisper-medium-fleurs-lang-id"
)

In [16]:
classifier(sample["audio"])

[{'score': 0.9999330043792725, 'label': 'Afrikaans'},
 {'score': 7.093003659974784e-06, 'label': 'Northern-Sotho'},
 {'score': 4.269145392754581e-06, 'label': 'Icelandic'},
 {'score': 3.266111207267386e-06, 'label': 'Danish'},
 {'score': 3.258066044509178e-06, 'label': 'Cantonese Chinese'}]

In [20]:
dataset = load_dataset("ashraq/esc50", split="train", streaming=True)
audio_sample = next(iter(dataset))["audio"]["array"]

Repo card metadata block was not found. Setting CardData to empty.


In [21]:
candidate_labels = ["Sound of a dog", "Sound of a vaccuum cleaner"]

In [22]:
classifier = pipeline(
    task="zero-shot-audio-classification", model="laion/clap-htsat-unfused"
)
classifier(audio_sample, candidate_labels=candidate_labels)

[{'score': 0.999738872051239, 'label': 'Sound of a dog'},
 {'score': 0.00026116592925973237, 'label': 'Sound of a vaccuum cleaner'}]

In [23]:
Audio(audio_sample, rate=16_000)